In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'
target_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = './log/style1/128/b0/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 1000
start_epoch = 0
train_batch = 200
test_batch = 200
lr = 0.1
schedule = [50, 250, 500, 750]
momentum = 0.1
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/128/b0/to_pggan/1000shot' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.5
cm_prob_init = 0.99
cm_prob_low = 0.01
cm_beta = 1.0

# augmentation
blur_prob = 0.2
blog_sig = 0.5
jpg_prob = 0.2

best_acc = 0

# sp
sp_alpha = 0.01
sp_beta = 0.01
fc_name = '_fc.'

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(source_dir, 'pggan/1000_shot_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/style1/128/b0/aug/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in teacher_model.parameters())/1000000.0))

    Total params: 4.01M


In [12]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(student_model.parameters())
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=10, after_scheduler=scheduler_cosine)

# Loss

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train Acc.', 'Valid Acc.', 'Source ACC.', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

In [14]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [15]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [16]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Train

In [17]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(model)
        loss_sp = reg_l2sp(model)
        loss =  loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
#         auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))
#         arc.update(auroc, inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
#         if batch_idx % 100 == 0:
#             print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
#                      batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    print('Train | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [18]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, targets.data)
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            top1.update(prec1[0], inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('Test | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [19]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc, train_auroc = train(train_loader, student_model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
    source_loss, source_acc, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)


    logger.append([state['lr'], train_loss, test_loss,  source_loss, train_acc, test_acc,source_acc, train_auroc, test_auroc, source_auroc])
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : student_model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 1000] LR: 0.100000
Train | 10/10 | Loss:1.5755 | MainLoss:1.5271 | SPLoss:0.5112 | CLSLoss:4.3294 | top1:49.6500 | AUROC:0.0000
Test | 161/10 | Loss:0.8168 | MainLoss:0.7687 | SPLoss:0.9394 | CLSLoss:3.8758 | top1:51.3676 | AUROC:0.5204
Test | 39/10 | Loss:0.3638 | MainLoss:0.3156 | SPLoss:0.9394 | CLSLoss:3.8758 | top1:87.2821 | AUROC:0.9997

Epoch: [2 | 1000] LR: 0.130000
Train | 10/10 | Loss:0.8132 | MainLoss:0.7657 | SPLoss:1.0242 | CLSLoss:3.7228 | top1:51.2000 | AUROC:0.0000
Test | 161/10 | Loss:0.7632 | MainLoss:0.7169 | SPLoss:1.0611 | CLSLoss:3.5767 | top1:52.2243 | AUROC:0.5324
Test | 39/10 | Loss:0.3721 | MainLoss:0.3257 | SPLoss:1.0611 | CLSLoss:3.5767 | top1:86.8846 | AUROC:0.9997

Epoch: [3 | 1000] LR: 0.160000
Train | 10/10 | Loss:0.7652 | MainLoss:0.7195 | SPLoss:1.1082 | CLSLoss:3.4631 | top1:52.4000 | AUROC:0.0000
Test | 161/10 | Loss:0.7440 | MainLoss:0.6987 | SPLoss:1.1934 | CLSLoss:3.3408 | top1:53.0187 | AUROC:0.5411
Test | 39/10 | Loss:0.4144 | MainL

Train | 10/10 | Loss:0.6278 | MainLoss:0.4673 | SPLoss:15.2810 | CLSLoss:0.7720 | top1:77.6500 | AUROC:0.0000
Test | 161/10 | Loss:0.3877 | MainLoss:0.2301 | SPLoss:14.9930 | CLSLoss:0.7707 | top1:95.6137 | AUROC:0.9909
Test | 39/10 | Loss:0.7670 | MainLoss:0.6094 | SPLoss:14.9930 | CLSLoss:0.7707 | top1:56.7436 | AUROC:0.9903

Epoch: [25 | 1000] LR: 0.399833
Train | 10/10 | Loss:0.6056 | MainLoss:0.4521 | SPLoss:14.5880 | CLSLoss:0.7649 | top1:78.3000 | AUROC:0.0000
Test | 161/10 | Loss:0.3722 | MainLoss:0.2213 | SPLoss:14.3280 | CLSLoss:0.7540 | top1:95.7446 | AUROC:0.9919
Test | 39/10 | Loss:0.7921 | MainLoss:0.6413 | SPLoss:14.3279 | CLSLoss:0.7540 | top1:55.3974 | AUROC:0.9895

Epoch: [26 | 1000] LR: 0.399807
Train | 10/10 | Loss:0.6100 | MainLoss:0.4632 | SPLoss:13.9443 | CLSLoss:0.7404 | top1:77.0500 | AUROC:0.0000
Test | 161/10 | Loss:0.3862 | MainLoss:0.2431 | SPLoss:13.5777 | CLSLoss:0.7254 | top1:95.4455 | AUROC:0.9926
Test | 39/10 | Loss:0.6760 | MainLoss:0.5330 | SPLoss:13

Train | 10/10 | Loss:0.5044 | MainLoss:0.4268 | SPLoss:7.2945 | CLSLoss:0.4662 | top1:80.3000 | AUROC:0.0000
Test | 161/10 | Loss:0.2745 | MainLoss:0.1976 | SPLoss:7.2009 | CLSLoss:0.4874 | top1:96.5857 | AUROC:0.9941
Test | 39/10 | Loss:0.7234 | MainLoss:0.6466 | SPLoss:7.2008 | CLSLoss:0.4874 | top1:57.2308 | AUROC:0.9893

Epoch: [48 | 1000] LR: 0.398722
Train | 10/10 | Loss:0.5110 | MainLoss:0.4352 | SPLoss:7.1070 | CLSLoss:0.4724 | top1:79.9000 | AUROC:0.0000
Test | 161/10 | Loss:0.2953 | MainLoss:0.2197 | SPLoss:7.1085 | CLSLoss:0.4487 | top1:96.3146 | AUROC:0.9937
Test | 39/10 | Loss:0.6785 | MainLoss:0.6029 | SPLoss:7.1085 | CLSLoss:0.4487 | top1:57.8974 | AUROC:0.9904

Epoch: [49 | 1000] LR: 0.398650
Train | 10/10 | Loss:0.4961 | MainLoss:0.4207 | SPLoss:7.0589 | CLSLoss:0.4745 | top1:80.0500 | AUROC:0.0000
Test | 161/10 | Loss:0.3074 | MainLoss:0.2329 | SPLoss:6.9778 | CLSLoss:0.4642 | top1:95.9097 | AUROC:0.9935
Test | 39/10 | Loss:0.6489 | MainLoss:0.5744 | SPLoss:6.9778 | C

Train | 10/10 | Loss:0.4955 | MainLoss:0.4237 | SPLoss:6.7148 | CLSLoss:0.4613 | top1:80.5000 | AUROC:0.0000
Test | 161/10 | Loss:0.2854 | MainLoss:0.2137 | SPLoss:6.7146 | CLSLoss:0.4590 | top1:96.4393 | AUROC:0.9940
Test | 39/10 | Loss:0.6783 | MainLoss:0.6065 | SPLoss:6.7146 | CLSLoss:0.4590 | top1:58.2436 | AUROC:0.9881

Epoch: [71 | 1000] LR: 0.039657
Train | 10/10 | Loss:0.4975 | MainLoss:0.4258 | SPLoss:6.7146 | CLSLoss:0.4573 | top1:80.3500 | AUROC:0.0000
Test | 161/10 | Loss:0.2799 | MainLoss:0.2080 | SPLoss:6.7247 | CLSLoss:0.4586 | top1:96.6449 | AUROC:0.9941
Test | 39/10 | Loss:0.6908 | MainLoss:0.6189 | SPLoss:6.7247 | CLSLoss:0.4586 | top1:57.5000 | AUROC:0.9884

Epoch: [72 | 1000] LR: 0.039646
Train | 10/10 | Loss:0.4791 | MainLoss:0.4074 | SPLoss:6.7187 | CLSLoss:0.4595 | top1:81.3000 | AUROC:0.0000
Test | 161/10 | Loss:0.2819 | MainLoss:0.2102 | SPLoss:6.7074 | CLSLoss:0.4637 | top1:96.5203 | AUROC:0.9940
Test | 39/10 | Loss:0.6851 | MainLoss:0.6134 | SPLoss:6.7074 | C

Train | 10/10 | Loss:0.4952 | MainLoss:0.4252 | SPLoss:6.5377 | CLSLoss:0.4604 | top1:79.9000 | AUROC:0.0000
Test | 161/10 | Loss:0.2823 | MainLoss:0.2123 | SPLoss:6.5461 | CLSLoss:0.4601 | top1:96.2710 | AUROC:0.9936
Test | 39/10 | Loss:0.6825 | MainLoss:0.6125 | SPLoss:6.5461 | CLSLoss:0.4601 | top1:58.3462 | AUROC:0.9882

Epoch: [94 | 1000] LR: 0.039340
Train | 10/10 | Loss:0.4906 | MainLoss:0.4206 | SPLoss:6.5450 | CLSLoss:0.4607 | top1:80.0000 | AUROC:0.0000
Test | 161/10 | Loss:0.2854 | MainLoss:0.2155 | SPLoss:6.5329 | CLSLoss:0.4571 | top1:96.1994 | AUROC:0.9935
Test | 39/10 | Loss:0.6755 | MainLoss:0.6056 | SPLoss:6.5329 | CLSLoss:0.4571 | top1:58.5385 | AUROC:0.9889

Epoch: [95 | 1000] LR: 0.039324
Train | 10/10 | Loss:0.4952 | MainLoss:0.4252 | SPLoss:6.5377 | CLSLoss:0.4577 | top1:80.3000 | AUROC:0.0000
Test | 161/10 | Loss:0.2856 | MainLoss:0.2158 | SPLoss:6.5246 | CLSLoss:0.4540 | top1:96.2243 | AUROC:0.9934
Test | 39/10 | Loss:0.6755 | MainLoss:0.6057 | SPLoss:6.5246 | C

...

Train | 10/10 | Loss:0.4824 | MainLoss:0.4138 | SPLoss:6.4259 | CLSLoss:0.4388 | top1:80.0500 | AUROC:0.0000
Test | 161/10 | Loss:0.2843 | MainLoss:0.2157 | SPLoss:6.4259 | CLSLoss:0.4388 | top1:95.7196 | AUROC:0.9922
Test | 39/10 | Loss:0.6836 | MainLoss:0.6149 | SPLoss:6.4259 | CLSLoss:0.4388 | top1:59.4615 | AUROC:0.9863

Epoch: [531 | 1000] LR: 0.000188
Train | 10/10 | Loss:0.4740 | MainLoss:0.4054 | SPLoss:6.4259 | CLSLoss:0.4388 | top1:80.8500 | AUROC:0.0000
Test | 161/10 | Loss:0.2843 | MainLoss:0.2157 | SPLoss:6.4260 | CLSLoss:0.4388 | top1:95.7196 | AUROC:0.9922
Test | 39/10 | Loss:0.6836 | MainLoss:0.6150 | SPLoss:6.4260 | CLSLoss:0.4388 | top1:59.4615 | AUROC:0.9860

Epoch: [532 | 1000] LR: 0.000187
Train | 10/10 | Loss:0.4631 | MainLoss:0.3945 | SPLoss:6.4260 | CLSLoss:0.4388 | top1:82.3000 | AUROC:0.0000
Test | 161/10 | Loss:0.2843 | MainLoss:0.2157 | SPLoss:6.4260 | CLSLoss:0.4388 | top1:95.7196 | AUROC:0.9922
Test | 39/10 | Loss:0.6836 | MainLoss:0.6149 | SPLoss:6.4260 |

Train | 10/10 | Loss:0.4668 | MainLoss:0.3982 | SPLoss:6.4266 | CLSLoss:0.4388 | top1:81.1000 | AUROC:0.0000
Test | 161/10 | Loss:0.2842 | MainLoss:0.2155 | SPLoss:6.4266 | CLSLoss:0.4388 | top1:95.7165 | AUROC:0.9920
Test | 39/10 | Loss:0.6840 | MainLoss:0.6154 | SPLoss:6.4266 | CLSLoss:0.4388 | top1:59.4103 | AUROC:0.9861

Epoch: [554 | 1000] LR: 0.000174
Train | 10/10 | Loss:0.4668 | MainLoss:0.3982 | SPLoss:6.4266 | CLSLoss:0.4388 | top1:81.3500 | AUROC:0.0000
Test | 161/10 | Loss:0.2842 | MainLoss:0.2155 | SPLoss:6.4266 | CLSLoss:0.4388 | top1:95.7165 | AUROC:0.9923
Test | 39/10 | Loss:0.6840 | MainLoss:0.6153 | SPLoss:6.4266 | CLSLoss:0.4388 | top1:59.4359 | AUROC:0.9862

Epoch: [555 | 1000] LR: 0.000173
Train | 10/10 | Loss:0.4787 | MainLoss:0.4100 | SPLoss:6.4266 | CLSLoss:0.4388 | top1:80.8500 | AUROC:0.0000
Test | 161/10 | Loss:0.2842 | MainLoss:0.2156 | SPLoss:6.4266 | CLSLoss:0.4388 | top1:95.7165 | AUROC:0.9923
Test | 39/10 | Loss:0.6839 | MainLoss:0.6152 | SPLoss:6.4265 |

Train | 10/10 | Loss:0.4742 | MainLoss:0.4056 | SPLoss:6.4269 | CLSLoss:0.4388 | top1:81.3500 | AUROC:0.0000
Test | 161/10 | Loss:0.2842 | MainLoss:0.2155 | SPLoss:6.4269 | CLSLoss:0.4388 | top1:95.7165 | AUROC:0.9923
Test | 39/10 | Loss:0.6840 | MainLoss:0.6153 | SPLoss:6.4269 | CLSLoss:0.4388 | top1:59.4359 | AUROC:0.9861

Epoch: [577 | 1000] LR: 0.000159
Train | 10/10 | Loss:0.4695 | MainLoss:0.4008 | SPLoss:6.4269 | CLSLoss:0.4388 | top1:81.5500 | AUROC:0.0000
Test | 161/10 | Loss:0.2842 | MainLoss:0.2155 | SPLoss:6.4269 | CLSLoss:0.4388 | top1:95.7165 | AUROC:0.9923
Test | 39/10 | Loss:0.6840 | MainLoss:0.6153 | SPLoss:6.4269 | CLSLoss:0.4388 | top1:59.4359 | AUROC:0.9859

Epoch: [578 | 1000] LR: 0.000159
Train | 10/10 | Loss:0.4753 | MainLoss:0.4066 | SPLoss:6.4269 | CLSLoss:0.4388 | top1:80.8500 | AUROC:0.0000
Test | 161/10 | Loss:0.2842 | MainLoss:0.2156 | SPLoss:6.4268 | CLSLoss:0.4388 | top1:95.7165 | AUROC:0.9922
Test | 39/10 | Loss:0.6839 | MainLoss:0.6153 | SPLoss:6.4268 |

Train | 10/10 | Loss:0.4711 | MainLoss:0.4025 | SPLoss:6.4265 | CLSLoss:0.4387 | top1:81.3000 | AUROC:0.0000
Test | 161/10 | Loss:0.2844 | MainLoss:0.2157 | SPLoss:6.4265 | CLSLoss:0.4387 | top1:95.7134 | AUROC:0.9922
Test | 39/10 | Loss:0.6836 | MainLoss:0.6150 | SPLoss:6.4265 | CLSLoss:0.4387 | top1:59.4615 | AUROC:0.9863

Epoch: [600 | 1000] LR: 0.000145
Train | 10/10 | Loss:0.4650 | MainLoss:0.3963 | SPLoss:6.4265 | CLSLoss:0.4387 | top1:82.4500 | AUROC:0.0000
Test | 161/10 | Loss:0.2844 | MainLoss:0.2157 | SPLoss:6.4265 | CLSLoss:0.4387 | top1:95.7165 | AUROC:0.9922
Test | 39/10 | Loss:0.6836 | MainLoss:0.6149 | SPLoss:6.4265 | CLSLoss:0.4387 | top1:59.4615 | AUROC:0.9862

Epoch: [601 | 1000] LR: 0.000145
Train | 10/10 | Loss:0.4705 | MainLoss:0.4018 | SPLoss:6.4265 | CLSLoss:0.4387 | top1:81.4500 | AUROC:0.0000
Test | 161/10 | Loss:0.2843 | MainLoss:0.2157 | SPLoss:6.4266 | CLSLoss:0.4387 | top1:95.7134 | AUROC:0.9922
Test | 39/10 | Loss:0.6837 | MainLoss:0.6150 | SPLoss:6.4265 |

Train | 10/10 | Loss:0.4706 | MainLoss:0.4020 | SPLoss:6.4268 | CLSLoss:0.4387 | top1:81.1500 | AUROC:0.0000
Test | 161/10 | Loss:0.2843 | MainLoss:0.2157 | SPLoss:6.4268 | CLSLoss:0.4387 | top1:95.7134 | AUROC:0.9923
Test | 39/10 | Loss:0.6837 | MainLoss:0.6151 | SPLoss:6.4268 | CLSLoss:0.4387 | top1:59.4487 | AUROC:0.9864

Epoch: [623 | 1000] LR: 0.000132
Train | 10/10 | Loss:0.4722 | MainLoss:0.4036 | SPLoss:6.4268 | CLSLoss:0.4387 | top1:81.0500 | AUROC:0.0000
Test | 161/10 | Loss:0.2843 | MainLoss:0.2157 | SPLoss:6.4268 | CLSLoss:0.4387 | top1:95.7134 | AUROC:0.9921
Test | 39/10 | Loss:0.6837 | MainLoss:0.6151 | SPLoss:6.4268 | CLSLoss:0.4387 | top1:59.4487 | AUROC:0.9861

Epoch: [624 | 1000] LR: 0.000131
Train | 10/10 | Loss:0.4588 | MainLoss:0.3901 | SPLoss:6.4268 | CLSLoss:0.4387 | top1:82.5000 | AUROC:0.0000
Test | 161/10 | Loss:0.2843 | MainLoss:0.2157 | SPLoss:6.4268 | CLSLoss:0.4387 | top1:95.7134 | AUROC:0.9923
Test | 39/10 | Loss:0.6837 | MainLoss:0.6151 | SPLoss:6.4268 |